# Developing an AI application

Going forward, AI algorithms will be incorporated into more and more everyday applications. For example, you might want to include an image classifier in a smart phone app. To do this, you'd use a deep learning model trained on hundreds of thousands of images as part of the overall application architecture. A large part of software development in the future will be using these types of models as common parts of applications. 

In this project, you'll train an image classifier to recognize different species of flowers. You can imagine using something like this in a phone app that tells you the name of the flower your camera is looking at. In practice you'd train this classifier, then export it for use in your application. We'll be using [this dataset](http://www.robots.ox.ac.uk/~vgg/data/flowers/102/index.html) of 102 flower categories, you can see a few examples below. 

<img src='assets/Flowers.png' width=500px>

The project is broken down into multiple steps:

* Load and preprocess the image dataset
* Train the image classifier on your dataset
* Use the trained classifier to predict image content

We'll lead you through each part which you'll implement in Python.

When you've completed this project, you'll have an application that can be trained on any set of labeled images. Here your network will be learning about flowers and end up as a command line application. But, what you do with your new skills depends on your imagination and effort in building a dataset. For example, imagine an app where you take a picture of a car, it tells you what the make and model is, then looks up information about it. Go build your own dataset and make something new.

First up is importing the packages you'll need. It's good practice to keep all the imports at the beginning of your code. As you work through this notebook and find you need to import a package, make sure to add the import up here.

In [1]:
import numpy as np
import json
import torchvision
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import torch
from torch import nn
from torch import optim
import torch.nn.functional as ff
import torchvision.models as model
from collections import OrderedDict
from torchvision import datasets,transforms,models

## Load the data

Here you'll use `torchvision` to load the data ([documentation](http://pytorch.org/docs/0.3.0/torchvision/index.html)). The data should be included alongside this notebook, otherwise you can [download it here](https://s3.amazonaws.com/content.udacity-data.com/nd089/flower_data.tar.gz). The dataset is split into three parts, training, validation, and testing. For the training, you'll want to apply transformations such as random scaling, cropping, and flipping. This will help the network generalize leading to better performance. You'll also need to make sure the input data is resized to 224x224 pixels as required by the pre-trained networks.

The validation and testing sets are used to measure the model's performance on data it hasn't seen yet. For this you don't want any scaling or rotation transformations, but you'll need to resize then crop the images to the appropriate size.

The pre-trained networks you'll use were trained on the ImageNet dataset where each color channel was normalized separately. For all three sets you'll need to normalize the means and standard deviations of the images to what the network expects. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`, calculated from the ImageNet images.  These values will shift each color channel to be centered at 0 and range from -1 to 1.
 

In [2]:
data_dir = 'flowers'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'
test_dir = data_dir + '/test'

In [3]:
# TODO: Define your transforms for the training, validation, and testing sets
train_data = transforms.Compose([transforms.RandomRotation(45),
                                      transforms.RandomResizedCrop(224),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])
test_data=transforms.Compose([transforms.Resize(256),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])
valid_data=transforms.Compose([transforms.Resize(256),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])

# TODO: Load the datasets with ImageFolder
traindf = datasets.ImageFolder(train_dir, transform = train_data)
testdf = datasets.ImageFolder(test_dir, transform = test_data)
validdf = datasets.ImageFolder(valid_dir, transform = valid_data)

# TODO: Using the image datasets and the trainforms, define the dataloader
traintfl = torch.utils.data.DataLoader(traindf, batch_size = 64, shuffle = True)
testtfl = torch.utils.data.DataLoader(testdf, batch_size = 64, shuffle = True)
validtfl = torch.utils.data.DataLoader(validdf, batch_size = 64, shuffle = True)

### Label mapping

You'll also need to load in a mapping from category label to category name. You can find this in the file `cat_to_name.json`. It's a JSON object which you can read in with the [`json` module](https://docs.python.org/2/library/json.html). This will give you a dictionary mapping the integer encoded categories to the actual names of the flowers.

In [4]:
import json

with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)

# Building and training the classifier

Now that the data is ready, it's time to build and train the classifier. As usual, you should use one of the pretrained models from `torchvision.models` to get the image features. Build and train a new feed-forward classifier using those features.

We're going to leave this part up to you. Refer to [the rubric](https://review.udacity.com/#!/rubrics/1663/view) for guidance on successfully completing this section. Things you'll need to do:

* Load a [pre-trained network](http://pytorch.org/docs/master/torchvision/models.html) (If you need a starting point, the VGG networks work great and are straightforward to use)
* Define a new, untrained feed-forward network as a classifier, using ReLU activations and dropout
* Train the classifier layers using backpropagation using the pre-trained network to get the features
* Track the loss and accuracy on the validation set to determine the best hyperparameters

We've left a cell open for you below, but use as many as you need. Our advice is to break the problem up into smaller parts you can run separately. Check that each part is doing what you expect, then move on to the next. You'll likely find that as you work through each part, you'll need to go back and modify your previous code. This is totally normal!

When training make sure you're updating only the weights of the feed-forward network. You should be able to get the validation accuracy above 70% if you build everything right. Make sure to try different hyperparameters (learning rate, units in the classifier, epochs, etc) to find the best model. Save those hyperparameters to use as default values in the next part of the project.

One last important tip if you're using the workspace to run your code: To avoid having your workspace disconnect during the long-running tasks in this notebook, please read in the earlier page in this lesson called Intro to
GPU Workspaces about Keeping Your Session Active. You'll want to include code from the workspace_utils.py module.

<font color='red'>**Note for Workspace users:** If your network is over 1 GB when saved as a checkpoint, there might be issues with saving backups in your workspace. Typically this happens with wide dense layers after the convolutional layers. If your saved checkpoint is larger than 1 GB (you can open a terminal and check with `ls -lh`), you should reduce the size of your hidden layers and train again.</font>

In [8]:
# TODO: Build and train your network
def nn_setup(structure = 'vgg16', lr = 0.001):
    
    model1 = models.vgg16(pretrained = True)
    for param in model1.parameters():
        param.requires_grad = False
       
    model1.classifier = nn.Sequential(OrderedDict([
                          ('c1', nn.Linear(25088, 2048)),
                          ('relu', nn.ReLU()),
                          ('c2', nn.Linear(2048, 256)),
                          ('relu', nn.ReLU()),
                          ('c3', nn.Linear(256, 102)),
                          ('o/p', nn.LogSoftmax(dim=1))
                          ]))
    model1 = model1.to('cuda')
    criterion = nn.NLLLoss()
    optimizer = optim.Adam(model1.classifier.parameters(), lr)
    
    return model1, criterion, optimizer




In [9]:
model1, criterion, optimizer = nn_setup()
print(model)

<module 'torchvision.models' from '/opt/conda/lib/python3.6/site-packages/torchvision-0.2.1-py3.6.egg/torchvision/models/__init__.py'>


In [10]:
n_epochs = 4
freq = 4
steps = 0
loss_show = []

for e in range(n_epochs):
    running_loss = 0
    for inputs, labels in traintfl:
        steps += 1
        
        inputs, labels = inputs.to('cuda'), labels.to('cuda')
        
        optimizer.zero_grad()
        
        outputs = model1.forward(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        if steps % freq == 0:
            model1.eval()
            vloss = 0
            accuracy = 0
            with torch.no_grad():
                for inputs, labels in validtfl:
                    inputs, labels = inputs.to('cuda'), labels.to('cuda')
                    
                    log_ps = model1.forward(inputs)
                    batch_loss = criterion(log_ps, labels)
                    vloss += batch_loss.item()
                    
                    ps = torch.exp(log_ps)
                    top_p, top_class = ps.topk(1, dim=1)
                    equals = top_class == labels.view(*top_class.shape)
                    accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
            print(f"Epoch {e+1}/{n_epochs}.. "
                  f"Loss: {running_loss/freq:.3f}.. "
                  f"VLoss: {vloss/len(validtfl):.3f}.. "
                  f"Acc: {accuracy/len(validtfl):.3f}")
            running_loss = 0
            model1.train()

Epoch 1/4.. Loss: 6.902.. VLoss: 5.316.. Acc: 0.028
Epoch 1/4.. Loss: 5.036.. VLoss: 4.600.. Acc: 0.118
Epoch 1/4.. Loss: 4.444.. VLoss: 3.942.. Acc: 0.144
Epoch 1/4.. Loss: 3.818.. VLoss: 3.582.. Acc: 0.250
Epoch 1/4.. Loss: 3.604.. VLoss: 3.331.. Acc: 0.252
Epoch 1/4.. Loss: 3.316.. VLoss: 2.775.. Acc: 0.401
Epoch 1/4.. Loss: 3.143.. VLoss: 2.488.. Acc: 0.446
Epoch 1/4.. Loss: 2.747.. VLoss: 2.269.. Acc: 0.488
Epoch 1/4.. Loss: 2.509.. VLoss: 2.238.. Acc: 0.450
Epoch 1/4.. Loss: 2.350.. VLoss: 1.860.. Acc: 0.514
Epoch 1/4.. Loss: 2.040.. VLoss: 1.637.. Acc: 0.569
Epoch 1/4.. Loss: 1.976.. VLoss: 1.642.. Acc: 0.565
Epoch 1/4.. Loss: 1.973.. VLoss: 1.508.. Acc: 0.565
Epoch 1/4.. Loss: 1.938.. VLoss: 1.353.. Acc: 0.635
Epoch 1/4.. Loss: 1.822.. VLoss: 1.350.. Acc: 0.626
Epoch 1/4.. Loss: 1.824.. VLoss: 1.303.. Acc: 0.639
Epoch 1/4.. Loss: 1.702.. VLoss: 1.185.. Acc: 0.666
Epoch 1/4.. Loss: 1.714.. VLoss: 1.085.. Acc: 0.688
Epoch 1/4.. Loss: 1.507.. VLoss: 1.102.. Acc: 0.685
Epoch 1/4.. 

## Testing your network

It's good practice to test your trained network on test data, images the network has never seen either in training or validation. This will give you a good estimate for the model's performance on completely new images. Run the test images through the network and measure the accuracy, the same way you did validation. You should be able to reach around 70% accuracy on the test set if the model has been trained well.

In [11]:
test_loss = 0
accuracy = 0
model1.to('cuda')

with torch.no_grad():
    for inputs, labels in testtfl:
        inputs, labels = inputs.to('cuda'), labels.to('cuda')
        
        log_ps = model1.forward(inputs)
        batch_loss = criterion(log_ps, labels)
                    
        test_loss += batch_loss.item()
                    
        ps = torch.exp(log_ps)
        top_p, top_class = ps.topk(1, dim=1)
        equals = top_class == labels.view(*top_class.shape)
        accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
        
print(f"Test accuracy: {accuracy/len(testtfl):.3f}")

Test accuracy: 0.842


## Save the checkpoint

Now that your network is trained, save the model so you can load it later for making predictions. You probably want to save other things such as the mapping of classes to indices which you get from one of the image datasets: `image_datasets['train'].class_to_idx`. You can attach this to the model as an attribute which makes inference easier later on.

```model.class_to_idx = image_datasets['train'].class_to_idx```

Remember that you'll want to completely rebuild the model later so you can use it for inference. Make sure to include any information you need in the checkpoint. If you want to load the model and keep training, you'll want to save the number of epochs as well as the optimizer state, `optimizer.state_dict`. You'll likely want to use this trained model in the next part of the project, so best to save it now.

In [39]:
# TODO: Save the checkpoint 
#model1.class_to_idx = train_data.class_to_idx
bundle={'pretrained_model': 'vgg16',
    'input_size': 25088,
    'output_size': 102,
    'learning_rate': 0.001,
    'classifier': model1.classifier,
    'epochs': n_epochs,
    'state_dict': model1.state_dict(),
    'optimizer' : optimizer.state_dict()
}

torch.save(bundle, 'vgg_checkpoint.pth')


## Loading the checkpoint

At this point it's good to write a function that can load a checkpoint and rebuild the model. That way you can come back to this project and keep working on it without having to retrain the network.

In [40]:
# TODO: Write a function that loads a checkpoint and rebuilds the model
def load_checkpoint(filepath):
    checkpoint = torch.load(filepath, map_location=lambda storage, loc: storage)
    _model = getattr(torchvision.models, checkpoint['pretrained_model'])(pretrained=True)
    _model.input_size = checkpoint['input_size']
    _model.output_size = checkpoint['output_size']
    _model.learning_rate = checkpoint['learning_rate']
    _model.classifier = checkpoint['classifier']
    _model.epochs = checkpoint['epochs']
    _model.load_state_dict(checkpoint['state_dict'])
    _model.optimizer = checkpoint['optimizer']
    return _model

loaded_model = load_checkpoint('vgg_checkpoint.pth')  
loaded_model


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (17): Conv2d

# Inference for classification

Now you'll write a function to use a trained network for inference. That is, you'll pass an image into the network and predict the class of the flower in the image. Write a function called `predict` that takes an image and a model, then returns the top $K$ most likely classes along with the probabilities. It should look like 

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

First you'll need to handle processing the input image such that it can be used in your network. 

## Image Preprocessing

You'll want to use `PIL` to load the image ([documentation](https://pillow.readthedocs.io/en/latest/reference/Image.html)). It's best to write a function that preprocesses the image so it can be used as input for the model. This function should process the images in the same manner used for training. 

First, resize the images where the shortest side is 256 pixels, keeping the aspect ratio. This can be done with the [`thumbnail`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) or [`resize`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) methods. Then you'll need to crop out the center 224x224 portion of the image.

Color channels of images are typically encoded as integers 0-255, but the model expected floats 0-1. You'll need to convert the values. It's easiest with a Numpy array, which you can get from a PIL image like so `np_image = np.array(pil_image)`.

As before, the network expects the images to be normalized in a specific way. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`. You'll want to subtract the means from each color channel, then divide by the standard deviation. 

And finally, PyTorch expects the color channel to be the first dimension but it's the third dimension in the PIL image and Numpy array. You can reorder dimensions using [`ndarray.transpose`](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.ndarray.transpose.html). The color channel needs to be first and retain the order of the other two dimensions.

In [43]:
def prepare_image(image):
    ''' Scales, crops, and normalizes a PIL image for a PyTorch model,
        returns an Numpy array
    '''
    image_pil = Image.open(image)
    image_transforms = transforms.Compose([transforms.Resize(256),
                                         transforms.CenterCrop(224),
                                         transforms.ToTensor(),
                                         transforms.Normalize([0.485, 0.456, 0.406],
                                                              [0.229, 0.224, 0.225])])
    
    image = image_transforms(image_pil)
    
    return image
    
prepare_image("flowers/test/10/image_07090.jpg")
    # TODO: Process a PIL image for use in a PyTorch model

tensor([[[-1.5357, -1.5357, -1.5185,  ..., -1.5357, -1.5185, -1.5185],
         [-1.5014, -1.5014, -1.5014,  ..., -1.5528, -1.5357, -1.5185],
         [-1.5014, -1.4843, -1.4672,  ..., -1.5699, -1.5528, -1.5528],
         ...,
         [-0.7308, -0.7308, -0.7308,  ..., -1.4500, -1.4500, -1.4329],
         [-0.7479, -0.7308, -0.7308,  ..., -1.4672, -1.4672, -1.4672],
         [-0.7993, -0.7993, -0.7993,  ..., -1.4500, -1.4500, -1.4500]],

        [[-0.7927, -0.7927, -0.7752,  ..., -1.0378, -1.0203, -1.0203],
         [-0.7577, -0.7577, -0.7577,  ..., -1.0903, -1.0728, -1.0553],
         [-0.7577, -0.7402, -0.7227,  ..., -1.1078, -1.0903, -1.0903],
         ...,
         [-0.3901, -0.4076, -0.4426,  ..., -0.7052, -0.7052, -0.6877],
         [-0.4076, -0.4251, -0.4601,  ..., -0.6877, -0.6877, -0.6877],
         [-0.4426, -0.4601, -0.4951,  ..., -0.6702, -0.6702, -0.6702]],

        [[-1.1421, -1.1421, -1.1247,  ..., -1.1247, -1.1073, -1.0898],
         [-1.1073, -1.1073, -1.1073,  ..., -1

To check your work, the function below converts a PyTorch tensor and displays it in the notebook. If your `process_image` function works, running the output through this function should return the original image (except for the cropped out portions).

In [ ]:
def imshow(image, ax=None, title=None):
    """Imshow for Tensor."""
    if ax is None:
        fig, ax = plt.subplots()
    
    # PyTorch tensors assume the color channel is the first dimension
    # but matplotlib assumes is the third dimension
    image = image.numpy().transpose((1, 2, 0))
    
    # Undo preprocessing
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = std * image + mean
    
    # Image needs to be clipped between 0 and 1 or it looks like noise when displayed
    image = np.clip(image, 0, 1)
    
    ax.imshow(image)
    
    return ax

## Class Prediction

Once you can get images in the correct format, it's time to write a function for making predictions with your model. A common practice is to predict the top 5 or so (usually called top-$K$) most probable classes. You'll want to calculate the class probabilities then find the $K$ largest values.

To get the top $K$ largest values in a tensor use [`x.topk(k)`](http://pytorch.org/docs/master/torch.html#torch.topk). This method returns both the highest `k` probabilities and the indices of those probabilities corresponding to the classes. You need to convert from these indices to the actual class labels using `class_to_idx` which hopefully you added to the model or from an `ImageFolder` you used to load the data ([see here](#Save-the-checkpoint)). Make sure to invert the dictionary so you get a mapping from index to class as well.

Again, this method should take a path to an image and a model checkpoint, then return the probabilities and classes.

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

In [ ]:
def predict(image_path, model1, topk=5):
    ''' Predict the class (or classes) of an image using a trained deep learning model.
    '''
    model1.to('cuda')
    model1.eval()
    img = prepare_image(image_path).numpy()
    img = torch.from_numpy(np.array([img])).float()

    with torch.no_grad():
        logps = model1.forward(img.cuda())
        
    ans = torch.exp(logps).data
    
    return ans.topk(topk)
    
   

In [ ]:
probs, classes = predict("flowers/test/10/image_07090.jpg", model1)
print (probs)
print (classes)

## Sanity Checking

Now that you can use a trained model for predictions, check to make sure it makes sense. Even if the testing accuracy is high, it's always good to check that there aren't obvious bugs. Use `matplotlib` to plot the probabilities for the top 5 classes as a bar graph, along with the input image. It should look like this:

<img src='assets/inference_example.png' width=300px>

You can convert from the class integer encoding to actual flower names with the `cat_to_name.json` file (should have been loaded earlier in the notebook). To show a PyTorch tensor as an image, use the `imshow` function defined above.

In [ ]:
# TODO: Display an image along with the top 5 classes

In [ ]:
plt.rcdefaults()
fig, ax = plt.subplots()
index = 1
path = test_dir + '/1/image_06743.jpg'
ps = predict(path, model1)
image = prepare_image(path)
ax1 = imshow(image, ax = plt)
ax1.title(cat_to_name[str(index)])
a1 = np.array(ps[0][0])
b1 = [cat_to_name[str(index+1)] for index in np.array(ps[1][0])]
fig,ax2 = plt.subplots(figsize=(5,5))
y_pos = np.arange(5)
ax2.set_yticks(y_pos)
ax2.set_yticklabels(b1)
ax2.set_xlabel('Prob %')
ax2.invert_yaxis()
ax2.barh(y_pos, a1)
plt.show()

<font color='red'>**Reminder for Workspace users:** If your network becomes very large when saved as a checkpoint, there might be issues with saving backups in your workspace. You should reduce the size of your hidden layers and train again. 
    
We strongly encourage you to delete these large interim files and directories before navigating to another page or closing the browser tab.</font>

In [ ]:
# TODO remove .pth files or move it to a temporary `~/opt` directory in this Workspace